In [1]:
import pandas as pd
import sqlalchemy
import pyodbc

engine = sqlalchemy.create_engine("mssql+pyodbc://localhost/transparencia?driver=ODBC+Driver+17+for+SQL+Server")
engine.connect()

SQL_Query = pd.read_sql_query(
'''select
[Número Licitação],
[Número Processo],
[Código Órgão],
[Código UG],
[Código Item Compra],
[CNPJ Participante],
[Flag Vencedor]
from [ParticipanteLicitacao]''', engine)

df = pd.DataFrame(SQL_Query, columns=['numero_licitacao','numero_processo','codigo_orgao','codigo_ug', 'codigo_item', 'cnpj', 'flag'], chunkzise=200)
df.info(verbose=True)

In [2]:
li = []
df = pd.read_csv('../DataSet/201903_ParticipantesLicitação.csv', index_col=None, header=0, low_memory=False, sep=';', encoding='latin-1')
li.append(df)
df = pd.read_csv('../DataSet/201904_ParticipantesLicitação.csv', index_col=None, header=0, low_memory=False, sep=';', encoding='latin-1')
li.append(df)
df = pd.read_csv('../DataSet/201905_ParticipantesLicitação.csv', index_col=None, header=0, low_memory=False, sep=';', encoding='latin-1')
li.append(df)

df = pd.concat(li, axis=0, ignore_index=True)

df.drop(columns=['Nome Órgão', 'Nome UG', 'Descrição Item Compra', 'Nome Participante'], inplace=True)
df.rename(columns={'Número Licitação':'numero_licitacao', 'Número Processo':'numero_processo', 'Código Órgão':'codigo_orgao', 'Código UG':'codigo_ug', 'Código Item Compra': 'item', 'CNPJ Participante':'cnpj', 'Flag Vencedor':'flag'}, inplace=True)

columns = [
'numero_licitacao',
'numero_processo',
'codigo_orgao',
'codigo_ug',
'item',
'cnpj',
'flag']

df.drop_duplicates(subset=columns, keep='first', inplace=True)

In [3]:
grouped = df.groupby(['numero_licitacao', 'item' ])

In [8]:
for name, group in grouped:
    #value = group['flag'].filter(lambda x: 'SIM' in x).count()
    #print(str(name) +' Wins: '+ str(value))
    #print(group['flag'].value_counts())
    value = len(group.loc[group.flag == 'SIM'])
    if(value > 1):    
        print(str(name) +' Wins: '+ str(value))

(12019, '0.') Wins: 21
(12019, '1605360500001201900026.') Wins: 2
(22019, '0.') Wins: 22
(32019, '0.') Wins: 25
(42019, '0.') Wins: 22
(52019, '0.') Wins: 12
(52019, '2570460500005201900013.') Wins: 2
(62019, '0.') Wins: 12
(72018, '0.') Wins: 2
(72019, '0.') Wins: 11
(72019, '7657200500007201900080.') Wins: 2
(82018, '1603520500008201800223.') Wins: 2
(82019, '0.') Wins: 10
(92019, '0.') Wins: 7
(102019, '0.') Wins: 8
(112019, '0.') Wins: 5
(122019, '0.') Wins: 5
(132019, '0.') Wins: 4
(142019, '0.') Wins: 3
(152019, '0.') Wins: 3
(162019, '0.') Wins: 6
(172019, '0.') Wins: 3
(182019, '0.') Wins: 3
(192019, '0.') Wins: 3
(202019, '0.') Wins: 2
(212019, '0.') Wins: 2
(222019, '0.') Wins: 2
(232019, '0.') Wins: 2
(242019, '0.') Wins: 2
(252019, '0.') Wins: 2
(282019, '0.') Wins: 2
(292019, '0.') Wins: 2
(322018, '1601060500032201800042.') Wins: 2


In [10]:
#Mais de 1 vencedor mesmo agrupado por licitação e item
check = [grouped.get_group((12019, '1605360500001201900026.')),
grouped.get_group((52019, '2570460500005201900013.')),
grouped.get_group((72019, '7657200500007201900080.')),
grouped.get_group((82018, '1603520500008201800223.')),
grouped.get_group((322018, '1601060500032201800042.'))
]

In [15]:
result = pd.concat(check, keys=['a', 'b', 'c', 'd', 'e'], sort=False)

In [17]:
result

numero_licitacao    numero_processo  codigo_orgao  codigo_ug  \
a 24937              12019  64123000322201963         52121     160536   
  24938              12019  64123000322201963         52121     160536   
  24939              12019  64123000322201963         52121     160536   
  24940              12019  64123000322201963         52121     160536   
  24941              12019  64123000322201963         52121     160536   
  24942              12019  64123000322201963         52121     160536   
b 132962             52019  25058000423201822         36000     257046   
  132963             52019  25058000423201822         36000     257046   
  132964             52019  25058000423201822         36000     257046   
  132965             52019  25058000423201822         36000     257046   
  132966             52019  25058000423201822         36000     257046   
  132967             52019  25058000423201822         36000     257046   
  132968             52019  25058000423201822         36000     257046   
  132969             52019  25058000423201822         36000     257046   
  132970             52019  25058000423201822         36000     257046   
  132971             52019  25058000423201822         36000     257046   
  132972             52019  25058000423201822         36000     257046   
  132973             52019  25058000423201822         36000     257046   
  132974             52019  25058000423201822         36000     257046   
c 711371             72019  63148004264201815         52131     765720   
  711372             72019  63148004264201815         52131     765720   
  711373             72019  63148004264201815         52131     765720   
d 723478             82018  80551005677201897         52121     160352   
  723479             82018  80551005677201897         52121     160352   
  723480             82018  80551005677201897         52121     160352   
  723481             82018  80551005677201897         52121     160352   
  723482             82018  80551005677201897         52121     160352   
  723483             82018  80551005677201897         52121     160352   
  723484             82018  80551005677201897         52121     160352   
  723485             82018  80551005677201897         52121     160352   
  723486             82018  80551005677201897         52121     160352   
  723487             82018  80551005677201897         52121     160352   
  723488             82018  80551005677201897         52121     160352   
e 927486            322018  00805960025372018         52121     160106   
  927487            322018  00805960025372018         52121     160106   
  927488            322018  00805960025372018         52121     160106   

                             item            cnpj flag  
a 24937   1605360500001201900026.  15735524000106  SIM  
  24938   1605360500001201900026.  14756414000150  NÃO  
  24939   1605360500001201900026.  31354105000172  NÃO  
  24940   1605360500001201900026.  23318893000113  SIM  
  24941   1605360500001201900026.  27204689000122  NÃO  
  24942   1605360500001201900026.  07734851000107  NÃO  
b 132962  2570460500005201900013.  05387567000186  NÃO  
  132963  2570460500005201900013.  19099184000117  SIM  
  132964  2570460500005201900013.  20470692000149  NÃO  
  132965  2570460500005201900013.  27184259000196  NÃO  
  132966  2570460500005201900013.  24101048000154  NÃO  
  132967  2570460500005201900013.  09319988000120  NÃO  
  132968  2570460500005201900013.  30807784000125  NÃO  
  132969  2570460500005201900013.  26503796000199  NÃO  
  132970  2570460500005201900013.  73261133000195  NÃO  
  132971  2570460500005201900013.  26507653000155  NÃO  
  132972  2570460500005201900013.  08961171000199  NÃO  
  132973  2570460500005201900013.  28302534000191  NÃO  
  132974  2570460500005201900013.  19571002000169  SIM  
c 711371  7657200500007201900080.  05638301000169  NÃO  
  711372  7657200500007201900080.  92629658000185  SIM  
  711373  7657200

Ivestigar se é possível obter 2 vencedores quando a quantidade de itens não é suficiente
Modalidade: Pregão - Registro de Preço

O pregão é uma modalidade de licitação, enquanto o registro de preços é um sistema de contratações. Isso significa que o pregão resulta num único contrato (ainda que possa ter a execução continuada), enquanto o registro de preços propicia uma série de contratações, respeitados os quantitativos máximos e a observância do período de um ano. Dito de outro modo, o pregão se exaure com uma única contratação, enquanto o registro de preços dá oportunidade a tantas contratações quantas forem possíveis (em face dos quantitativos máximos licitados e do prazo de validade).

http://www.portaltransparencia.gov.br/entenda-a-gestao-publica/licitacoes-e-contratacoes

https://www.direitonet.com.br/artigos/exibir/6351/Registro-de-precos-uma-alternativa-inteligente-para-economizar

https://triunfolegis.jusbrasil.com.br/artigos/401055684/modalidades-de-licitacao-concorrencia

https://www.rcc.com.br/blog/licitacao-pregao-o-que-ainda-preciso-saber-sobre-essa-modalidade/

https://www.rcc.com.br/blog/o-que-e-dispensa-de-licitacao-2/

https://www.rcc.com.br/blog/inexigibilidade-de-licitacao/

In [8]:
select = []
count = 0
for name, group in grouped:
    #value = group['flag'].filter(lambda x: 'SIM' in x).count()
    #print(str(name) +' Wins: '+ str(value))
    #print(group['flag'].value_counts())
    sim = len(group.loc[group.flag == 'SIM'])
    nao = len(group.loc[group.flag == 'NÃO'])
    if(sim == 1 and nao >= 1):    
        count+=1
        select.append(group)

In [10]:
print(str(count))
result = pd.concat(select, sort=False)

140956


TypeError: first() missing 1 required positional argument: 'offset'

In [11]:
result

,numero_licitacao,numero_processo,codigo_orgao,codigo_ug,item,cnpj,flag
1,12017,2016384880,93420,925989,9259890500001201700001.,26124481000130,NÃO
2,12017,2016384880,93420,925989,9259890500001201700001.,07675303000153,SIM
3,12017,2016384880,93420,925989,9259890500001201700001.,17807338000152,NÃO
4,12017,2016384880,93420,925989,9259890500001201700001.,06061654000101,NÃO
5,12017,2016384880,93420,925989,9259890500001201700001.,22773778000176,NÃO
6,12017,2016384880,93420,925989,9259890500001201700001.,03489595000189,NÃO
7,12017,2016384880,93420,925989,9259890500001201700001.,24857936000100,NÃO
458354,12018,23072031421201871,26238,153258,1532580500001201800001.,02421421000111,NÃO
458355,12018,23072031421201871,26238,153258,1532580500001201800001.,02558157000162,NÃO
458356,12018,23072031421201871,26238,153258,1532580500001201800001.,05423963000111,NÃO


In [14]:
test = result.groupby(['numero_licitacao', 'item'])['cnpj'].apply(';'.join).reset_index()

In [15]:
test

,numero_licitacao,item,cnpj
0,12017,9259890500001201700001.,26124481000130;07675303000153;17807338000152;0...
1,12018,1532580500001201800001.,02421421000111;02558157000162;05423963000111;4...
2,12018,1532580500001201800002.,02421421000111;05423963000111;40432544000147;0...
3,12018,1545820500001201800001.,18261811000101;09509234000133;09239494000136;0...
4,12018,1552160500001201800001.,22064428000130;04246291000153;04428101000119;0...
5,12018,1552160500001201800002.,00881764000133;14234924000167;04246291000153;2...
6,12018,1552160500001201800003.,04246291000153;22064428000130;14234924000167;0...
7,12018,1552160500001201800007.,04428101000119;14234924000167;00881764000133;2...
8,12018,1552160500001201800008.,04246291000153;11292106000122;14234924000167;0...
9,12018,1552160500001201800011.,00881764000133;14234924000167;22064428000130;0...


In [22]:
result["event"] = result["numero_licitacao"].map(str) + result["item"]
result

,numero_licitacao,numero_processo,codigo_orgao,codigo_ug,item,cnpj,flag,event
1,12017,2016384880,93420,925989,9259890500001201700001.,26124481000130,NÃO,120179259890500001201700001.
2,12017,2016384880,93420,925989,9259890500001201700001.,07675303000153,SIM,120179259890500001201700001.
3,12017,2016384880,93420,925989,9259890500001201700001.,17807338000152,NÃO,120179259890500001201700001.
4,12017,2016384880,93420,925989,9259890500001201700001.,06061654000101,NÃO,120179259890500001201700001.
5,12017,2016384880,93420,925989,9259890500001201700001.,22773778000176,NÃO,120179259890500001201700001.
6,12017,2016384880,93420,925989,9259890500001201700001.,03489595000189,NÃO,120179259890500001201700001.
7,12017,2016384880,93420,925989,9259890500001201700001.,24857936000100,NÃO,120179259890500001201700001.
458354,12018,23072031421201871,26238,153258,1532580500001201800001.,02421421000111,NÃO,120181532580500001201800001.
458355,12018,23072031421201871,26238,153258,1532580500001201800001.,02558157000162,NÃO,120181532580500001201800001.
458356,12018,23072031421201871,26238,153258,1532580500001201800001.,05423963000111,NÃO,120181532580500001201800001.


In [23]:
path = 'C:\\Users\\leosm\\Documents\\Projects\\TCC\\DataSet\\results\\'
result.to_csv(path+'result.csv', index=False, sep=';', header=True, encoding='latin-1')
result.drop(columns=['numero_licitacao', 'item', 'numero_processo','codigo_orgao','codigo_ug', 'flag']).to_csv(path+'resultFiltered.csv', index=False, sep=';', header=True, encoding='latin-1')